In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

import socketio

from flask import Flask
from flask_socketio import SocketIO, send, emit

In [2]:
sio = socketio.Client()

app = Flask(__name__)
app.config['SECRET_KEY'] = 'secret!'
server = SocketIO(app, logger=True, engineio_logger=True)

Server initialized for threading.


In [3]:
@server.on('data', namespace="/recv_data")
def got_data(data):
    print("Received data in part2")
    inpt = tf.convert_to_tensor(data)
    out = model(inpt).numpy().tolist()
    sio.emit('data', data=out, namespace="/recv_data")

@server.on('connect', namespace="/recv_model")
def connect(auth):
    print("Dispatcher Connected, auth:", auth["name"])

@server.on('disconnect')
def disconnect():
    print("Dispatcher disconnected")

@server.on('connect', namespace="/recv_data")
def connect():
    print("Previous node connected")

@server.on('dispatch', namespace="/recv_model")
def recv_model(model_json, next_node):
    global model
    model = tf.keras.models.model_from_json(model_json)
    print(model_json)
    #model.save("models/part2")
    print(next_node)
    sio.connect(next_node, namespaces='/recv_data')

In [4]:
if __name__ == '__main__':
    server.run(app, port=5002, use_reloader=False)

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5002/ (Press CTRL+C to quit)
-qBWf6weTvGbp7x9AAAA: Sending packet OPEN data {'sid': '-qBWf6weTvGbp7x9AAAA', 'upgrades': ['websocket'], 'pingTimeout': 20000, 'pingInterval': 25000}
127.0.0.1 - - [26/Sep/2021 23:58:48] "GET /socket.io/?transport=polling&EIO=4&t=1632725928.1845448 HTTP/1.1" 200 -
-qBWf6weTvGbp7x9AAAA: Received request to upgrade to websocket
-qBWf6weTvGbp7x9AAAA: Upgrade to websocket successful
-qBWf6weTvGbp7x9AAAA: Received packet MESSAGE data 0/recv_model,{"name":"dispatcher"}
-qBWf6weTvGbp7x9AAAA: Sending packet MESSAGE data 0/recv_model,{"sid":"6l1lO-7c30D9jFDrAAAB"}
-qBWf6weTvGbp7x9AAAA: Received packet MESSAGE data 2/recv_model,["dispatch","{\"class_name\": \"Functional\", \"config\": {\"name\": \"model_1\", \"layers\": [{\"class_name\": \"InputLayer\", \"config\": {\"batch_input_shape\": [3, 3], \"dtype\": \"float32\", \"sparse\": false, \"ragged\": false, \"name\": \"input_2\"}, \"name\": \"input_2\", \"inbound_nodes\": []}, {\"class

Dispatcher Connected, auth: dispatcher


0_7MKMzzvhpHumoqAAAC: Sending packet OPEN data {'sid': '0_7MKMzzvhpHumoqAAAC', 'upgrades': ['websocket'], 'pingTimeout': 20000, 'pingInterval': 25000}
127.0.0.1 - - [26/Sep/2021 23:58:48] "GET /socket.io/?transport=polling&EIO=4&t=1632725928.6443276 HTTP/1.1" 200 -
0_7MKMzzvhpHumoqAAAC: Received request to upgrade to websocket
0_7MKMzzvhpHumoqAAAC: Upgrade to websocket successful
0_7MKMzzvhpHumoqAAAC: Received packet MESSAGE data 0/recv_data,
0_7MKMzzvhpHumoqAAAC: Sending packet MESSAGE data 0/recv_data,{"sid":"jkXtvEZivwhGUfzTAAAD"}


{"class_name": "Functional", "config": {"name": "model_1", "layers": [{"class_name": "InputLayer", "config": {"batch_input_shape": [3, 3], "dtype": "float32", "sparse": false, "ragged": false, "name": "input_2"}, "name": "input_2", "inbound_nodes": []}, {"class_name": "Dense", "config": {"name": "dense_1", "trainable": true, "dtype": "float32", "units": 1, "activation": "linear", "use_bias": true, "kernel_initializer": {"class_name": "Constant", "config": {"value": 1.0}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_constraint": null, "bias_constraint": null}, "name": "dense_1", "inbound_nodes": [[["input_2", 0, 0, {}]]]}], "input_layers": [["input_2", 0, 0]], "output_layers": [["dense_1", 0, 0]]}, "keras_version": "2.6.0", "backend": "tensorflow"}
http://127.0.0.1:5000/
Previous node connected


0_7MKMzzvhpHumoqAAAC: Received packet MESSAGE data 2/recv_data,["data",[[3.0,3.0,3.0],[7.0,7.0,7.0],[11.0,11.0,11.0]]]
received event "data" from jkXtvEZivwhGUfzTAAAD [/recv_data]


Received data in part2


-qBWf6weTvGbp7x9AAAA: Sending packet PING data None
0_7MKMzzvhpHumoqAAAC: Sending packet PING data None
0_7MKMzzvhpHumoqAAAC: Received packet PONG data 
-qBWf6weTvGbp7x9AAAA: Unexpected error "", closing connection
127.0.0.1 - - [26/Sep/2021 23:59:35] "GET /socket.io/?transport=websocket&EIO=4&sid=-qBWf6weTvGbp7x9AAAA&t=1632725928.198545 HTTP/1.1" 200 -
